## notMNIST - CNN


In [1]:
from __future__ import print_function

import numpy as np
from scipy import ndimage
import pickle

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import advanced_activations
from keras.layers.normalization import BatchNormalization
from keras import backend as K

import tensorflow as tf

# Checking tensorflow processing devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7945318862555976339
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 1526107340
locality {
  bus_id: 1
}
incarnation: 1127238059845905014
physical_device_desc: "device: 0, name: GeForce GTX 680, pci bus id: 0000:01:00.0"
]


### Data Loading

In [2]:
pickle_file = './notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f,encoding='iso-8859-1')
    X_train = save['train_dataset']
    y_train = save['train_labels']
    X_validation = save['valid_dataset']
    y_validation = save['valid_labels']
    X_test = save['test_dataset']
    y_test = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', X_train.shape, y_train.shape)
    print('Validation set', X_validation.shape, y_validation.shape)
    print('Test set', X_test.shape, y_test.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_validation = X_validation.reshape(X_validation.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_validation = X_validation.reshape(X_validation.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_validation /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_validation shape:', X_validation.shape)
print('y_validation shape:', y_validation.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)
print(X_train.shape[0], 'Train samples')
print(X_validation.shape[0], 'Validation samples')
print(X_test.shape[0], 'Test samples')

X_train shape: (200000, 28, 28, 1)
y_train shape: (200000,)
X_validation shape: (10000, 28, 28, 1)
y_validation shape: (10000,)
X_test shape: (10000, 28, 28, 1)
y_test shape: (10000,)
200000 Train samples
10000 Validation samples
10000 Test samples


### Data Augmentation

In [ ]:
# Augment training data
def augment_training_data(images, labels):
    """
    Generates

    Creates an additional 300,000 
    
    Takes ~1.25 minutes with an i7/16gb machine
    """

    # Empty lists to fill
    expanded_images = []
    expanded_labels = []

    # Looping through
    j = 0   # counter
    for x, y in zip(images, labels):
        j = j + 1
        if j % 10000 == 0:
            print('Expanding data: %03d / %03d' % (j, np.size(images, 0)))

        # register original data
        expanded_images.append(x)
        expanded_labels.append(y)

        # get a value for the background
        # zero is the expected value, but median() is used to estimate background's value
        bg_value = np.median(x)  # this is regarded as background's value
        image = np.reshape(x, (-1, 28))

        for i in range(4):
            # rotate the image with random degree
            angle = np.random.randint(-15, 15, 1)
            new_img = ndimage.rotate(
                image, angle, reshape=False, cval=bg_value)

            # shift the image with random distance
            shift = np.random.randint(-2, 2, 2)
            new_img_ = ndimage.shift(new_img, shift, cval=bg_value)

            # register new training data
            expanded_images.append(np.reshape(new_img_, (28, 28, 1)))
            expanded_labels.append(y)

    # images and labels are concatenated for random-shuffle at each epoch
    # notice that pair of image and label should not be broken
#     expanded_train_total_data = np.concatenate((expanded_images, expanded_labels), axis=1)
#     print(np.array(expanded_images).shape)
#     print(np.array(expanded_labels).shape)
#     np.random.shuffle(expanded_train_total_data)

#     return expanded_train_total_data
    return expanded_images, expanded_labels


print('Starting')
augmented = augment_training_data(X_train, y_train)
print('Completed')

# Appending to the end of the current X/y train
X_train_aug = np.append(X_train, augmented[0], axis=0)
y_train_aug = np.append(y_train, augmented[1])

# Saving as a numpy array
print('\nSaving NP arrays')
np.save('X_train_augmented.npy', X_train)
np.save('y_train_augmented.npy', y_train)
print('Completed')

print('\nX_train shape:', X_train_aug.shape)
print('y_train shape:', y_train_aug.shape)
print(X_train_aug.shape[0], 'Train samples')
print(X_validation.shape[0], 'Validation samples')
print(X_test.shape[0], 'Test samples')

#### Loading In Augmented Data

If not running the data augmentation code above

In [4]:
# Uncomment if not performing data augmentation here
X_train_aug = np.load('X_train_augmented.npy')
y_train_aug = np.load('y_train_augmented.npy')

print('X_train_aug shape:', X_train_aug.shape)
print('y_train_aug shape:', y_train_aug.shape)
print(X_train_aug.shape[0], 'Train samples')
print(X_validation.shape[0], 'Validation samples')
print(X_test.shape[0], 'Test samples')

X_train_aug shape: (1200000, 28, 28, 1)
y_train_aug shape: (1200000,)
1200000 Train samples
10000 Validation samples
10000 Test samples


### Model Training on Augmented Data

In [5]:
# One-hot encoding for keras input
num_classes = 10
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_train_aug = keras.utils.np_utils.to_categorical(y_train_aug, num_classes)
y_validation = keras.utils.np_utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [ ]:
batch_size = 128
epochs = 20

## Constructing the model
model = Sequential()

# First hidden layer
model.add(Conv2D(32, kernel_size=(5, 5),
                 input_shape=input_shape))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second hidden layer
model.add(Conv2D(64, (5, 5)))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third hidden layer
model.add(Flatten())
model.add(Dense(1024))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train_aug, y_train_aug,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validation, y_validation))

score = model.evaluate(X_test, y_test, verbose=0)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Train on 1200000 samples, validate on 10000 samples
Epoch 1/20
1200000/1200000 [==============================] - 206s - loss: 0.4686 - acc: 0.8629 - val_loss: 0.4485 - val_acc: 0.8646
Epoch 2/20
1200000/1200000 [==============================] - 206s - loss: 0.3713 - acc: 0.8903 - val_loss: 0.4509 - val_acc: 0.8586
Epoch 3/20
1200000/1200000 [==============================] - 206s - loss: 0.3439 - acc: 0.8978 - val_loss: 0.3027 - val_acc: 0.9093
Epoch 4/20
1200000/1200000 [==============================] - 206s - loss: 0.3276 - acc: 0.9024 - val_loss: 0.3370 - val_acc: 0.9061
Epoch 5/20
1200000/1200000 [==============================] - 205s - loss: 0.3153 - acc: 0.9055 - val_loss: 0.3041 - val_acc: 0.9153
Epoch 6/20
1200000/1200000 [==============================] - 204s - loss: 0.3063 - acc: 0.9081 - val_loss: 0.2942 - val_acc: 0.9151
Epoch 7/20
1200000/1200000 [==============================] - 206s - loss: 0.2990 - acc: 0.9101 - val_loss: 0.2831 - val_acc: 0.9168
Epoch 8/20
120000

### Model Training on non-Augmented Data

In [19]:
batch_size = 128
epochs = 20

## Constructing the model
model = Sequential()

# First hidden layer
model.add(Conv2D(32, kernel_size=(5, 5),
                 input_shape=input_shape))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second hidden layer
model.add(Conv2D(64, (5, 5)))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third hidden layer
model.add(Flatten())
model.add(Dense(1024))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validation, y_validation))

score = model.evaluate(X_test, y_test, verbose=0)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])


# 0.9626 with batch norm on second layer only
# Note: Batch norm on the first layer tends to have poor performance for some reason

Train on 200000 samples, validate on 10000 samples
Epoch 1/20
200000/200000 [==============================] - 33s - loss: 0.5558 - acc: 0.8405 - val_loss: 0.4321 - val_acc: 0.8739
Epoch 2/20
200000/200000 [==============================] - 32s - loss: 0.4045 - acc: 0.8834 - val_loss: 0.4068 - val_acc: 0.8799
Epoch 3/20
200000/200000 [==============================] - 33s - loss: 0.3720 - acc: 0.8922 - val_loss: 0.3950 - val_acc: 0.8823
Epoch 4/20
200000/200000 [==============================] - 33s - loss: 0.3509 - acc: 0.8973 - val_loss: 0.6630 - val_acc: 0.8100
Epoch 5/20
200000/200000 [==============================] - 33s - loss: 0.3348 - acc: 0.9023 - val_loss: 0.3453 - val_acc: 0.8974
Epoch 6/20
200000/200000 [==============================] - 33s - loss: 0.3231 - acc: 0.9055 - val_loss: 0.4066 - val_acc: 0.8802
Epoch 7/20
200000/200000 [==============================] - 34s - loss: 0.3138 - acc: 0.9082 - val_loss: 0.3538 - val_acc: 0.8932
Epoch 8/20
200000/200000 [=============

### Model Training - Different Architectures w/ out augmentation

In [14]:
# Simple convnet achieved 98.19% on test, code here: https://github.com/alex-petrenko/udacity-deep-learning/blob/14714ee4151b798cde0a31a94ac65e08b87d0f65/assignment_04_convolutions.py#L39
# (5,5)->(5,5)->pool->(3,3)->(3,3)->pool->fc1024->fc1024->logits
# INFO Starting new epoch #121!
# INFO Minibatch loss: 0.150696, reg loss: 0.041653, accuracy: 96.88%
# INFO Train loss: 0.068505, train accuracy: 99.28%
# INFO Test loss: 0.118685, TEST ACCURACY: 98.05% BEST ACCURACY 98.19% <<<<<<<
batch_size = 128
epochs = 20

model = Sequential()

# Layer 1
model.add(Conv2D(32, kernel_size=(5, 5), input_shape=input_shape))
model.add(advanced_activations.LeakyReLU(alpha=0.03))
# model.add(BatchNormalization(axis=1))

# Layer 2
model.add(Conv2D(32, kernel_size=(5, 5)))
model.add(advanced_activations.LeakyReLU(alpha=0.03))
# model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Layer 3
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(advanced_activations.LeakyReLU(alpha=0.03))
model.add(BatchNormalization(axis=1))

# Layer 4
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(advanced_activations.LeakyReLU(alpha=0.03))
# model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Layer 5
model.add(Flatten())
model.add(Dense(1024, activation='relu'))

# Layer 6
model.add(Dense(1024, activation='relu'))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validation, y_validation))

score = model.evaluate(X_test, y_test, verbose=0)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Train on 200000 samples, validate on 10000 samples
Epoch 1/20
200000/200000 [==============================] - 204s - loss: 0.7616 - acc: 0.7603 - val_loss: 0.5280 - val_acc: 0.8443
Epoch 2/20
200000/200000 [==============================] - 202s - loss: 0.5012 - acc: 0.8477 - val_loss: 0.4010 - val_acc: 0.8749
Epoch 3/20
200000/200000 [==============================] - 202s - loss: 0.4381 - acc: 0.8664 - val_loss: 0.3666 - val_acc: 0.8876
Epoch 4/20
200000/200000 [==============================] - 201s - loss: 0.4018 - acc: 0.8770 - val_loss: 0.3533 - val_acc: 0.8929
Epoch 5/20
200000/200000 [==============================] - 201s - loss: 0.3774 - acc: 0.8844 - val_loss: 0.3173 - val_acc: 0.8971
Epoch 6/20
200000/200000 [==============================] - 202s - loss: 0.3605 - acc: 0.8894 - val_loss: 0.3047 - val_acc: 0.9051
Epoch 7/20
200000/200000 [==============================] - 202s - loss: 0.3446 - acc: 0.8935 - val_loss: 0.3002 - val_acc: 0.9072
Epoch 8/20
200000/200000 [======

Perf without augmented dataset and on 20 epochs:

- all batch normalization: 0.8766
- no batch normalization: 0.9647(?)
- batch normalization on the second layer only: 0.964
- batch normalization on the third layer (w/ axis=1): 0.9701

In [ ]:
# Test accuracy: 98.0%
# Implementation:
# 2 CNNs with max pooling followed by a 1 layer fully-connected NN:
# Patch size = 5x5
# Stride for CNN = 1
# Size of pooling size = 2x2
# Stride for Pooling = 2
# Depth = 50, 100
# Hidden layer Nodes in FCNN = 512
# Dropout_keep_probe = init: 95%, decaying to 70% 

In [15]:
# est accuracy: 97.2%
# Implementation:
# 2 CNNs with max pooling followed by a 1 layer fully-connected NN:
# Patch size = 7x7
# Stride for CNN = 1
# Size of pooling size = 2x2
# Stride for Pooling = 2
# Depth = 50, 100
# Final layer nodes = 512
# Dropout_keep_probe = 0.7 